In [1]:
import pyspark.sql.functions as F
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
import pandas as pd

from feature_engineering.engineering import engineerFeatures
from utils.utils import gapfilling, serialize
from modelling.model_utils import splitData, prepare_data, train_model, MLFlow_train_model
import mlflow

In [2]:
spark = SparkSession\
            .builder\
            .appName("test-app")\
            .getOrCreate()

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

/usr/local/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/30 15:26:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# [TODO]: move config dictionnaries to a json config file
features_config = {

    "discount_rate":{},
    "promoted_percent":{"promoted_hierarchy": "sku", "group_key":"subclass"},
    "week_of_year":{},
    "promo_category":{},
}

model_config={
    "model": "xgboost",
    "params": {},
    "hierarchy_columns": ["sku", "subclass", "store_id", "region_id"],
    "target": "units",
    "train_startDate": "2018-01-01",
    "train_endDate": "2020-01-01",
    "inference_startDate": "2019-11-01",
    "inference_endDate": "2020-12-21",
}

path = "data/"

In [6]:
# Union transcational data

schema = StructType(
    [StructField("customer_id", StringType(), True),
    StructField("week_index", StringType(), True),
    StructField("sku", StringType(), True),
    StructField("promo_cat", StringType(), True),
    StructField("discount", FloatType(), True),
    StructField("store_id", StringType(), True)],
)

transactions = spark.read.csv(
    "data/transactions_*.csv", 
    schema=schema,
    header=False
)

In [8]:
# Reading data
customers = spark.read.csv(
    "data/customers.csv", 
    header="true", 
    inferSchema="true")

calendar = spark.read.csv(
    "data/calendar.csv", 
    header="true", 
    inferSchema="true")

products = spark.read.csv(
    "data/products.csv", 
    header="true", 
    inferSchema="true")

stores = spark.read.csv(
    "data/stores.csv", 
    header="true", 
    inferSchema="true")

In [9]:
# Adding location hierarchy for customers

customers = customers.select(
    F.col("customer_id").cast("string"), 
    F.col("store_pref").cast("string").alias("store_id")
)

stores = stores.select(
    F.col("store_id").cast("string"), 
    F.col("store_region").cast("string").alias("region_id")
).dropDuplicates()

In [10]:
products = products.select(
    F.col("prod_id").cast("string").alias("sku"),
    F.col("prod_subclass").cast("string").alias("subclass"),
    F.col("prod_class").cast("string").alias("class"),
    F.col("prod_dept").cast("string").alias("dept"),
    F.col("prod_base_price").cast("float").alias("base_price"),
).dropDuplicates()

products.show(5)

+---+--------+-----+----+----------+
|sku|subclass|class|dept|base_price|
+---+--------+-----+----+----------+
|271|       0|    0|   7|    142.86|
|289|       0|    0|   3|      20.0|
|369|       0|    0|   2|     16.67|
|410|       1|    1|  14|    525.33|
|559|       0|    0|   4|     48.57|
+---+--------+-----+----+----------+
only showing top 5 rows



In [12]:
# daily calendar -> weekly calendar
weekly_calendar = calendar.where(
    F.col("day_of_week")=="0"
).select(
    F.to_date(F.col("calendar_day"),"MM-dd-yy").alias("date")
).distinct(
).sort(
    F.col("date").asc()
).withColumn(
    "week_index", F.monotonically_increasing_id()
).select(
    F.col("week_index").cast("string"),
    F.col("date")
)
weekly_calendar.show(5)

+----------+----------+
|week_index|      date|
+----------+----------+
|         0|2018-01-01|
|         1|2018-01-08|
|         2|2018-01-15|
|         3|2018-01-22|
|         4|2018-01-29|
+----------+----------+
only showing top 5 rows



In [14]:
# add hierarchies
demand_data = transactions.groupby(
    "sku", "store_id", "week_index"
).agg(
    F.count("*").alias("units"),
    F.first("promo_cat").alias("promo_cat"),
    F.max("discount").alias("discount"),
).join(
    weekly_calendar, on=["week_index"], how="inner"
).drop("week_index")

serialize(spark, demand_data, path + "demand_data.parquet").show(5)


22/12/28 17:44:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/12/28 17:44:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/12/28 17:44:31 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


+---+--------+-----+---------+--------+----------+
|sku|store_id|units|promo_cat|discount|      date|
+---+--------+-----+---------+--------+----------+
|  0|      87|    1|      nan|    null|2020-02-03|
|100|      83|    1|      nan|    null|2020-02-03|
|101|      24|    1|      nan|    null|2020-02-03|
|101|      55|    1|      nan|    null|2020-02-03|
|102|      40|    1|      nan|    null|2020-02-03|
+---+--------+-----+---------+--------+----------+
only showing top 5 rows



In [16]:
demand_data = spark.read.parquet(path + "demand_data.parquet", header="true", inferSchema="true")

sales_filled_data = gapfilling(demand_data, date_column="date", product_column="sku", location_column="store_id")

serialize(spark, sales_filled_data, path + "sales_filled_data.parquet").show(5)


22/12/28 17:44:59 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/12/28 17:44:59 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/12/28 17:45:02 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


+----------+---+--------+-----+---------+--------+
|      date|sku|store_id|units|promo_cat|discount|
+----------+---+--------+-----+---------+--------+
|2018-01-01|  0|      10|    0|     null|    null|
|2018-01-01|  0|      20|    0|     null|    null|
|2018-01-01|  0|      29|    0|     null|    null|
|2018-01-01|  0|      34|    0|     null|    null|
|2018-01-01|  0|      41|    0|     null|    null|
+----------+---+--------+-----+---------+--------+
only showing top 5 rows



In [18]:
# Adding product and location hierarchies to demand data
demand_data = spark.read.parquet(path + "sales_filled_data.parquet", header="true", inferSchema="true")

sales_data = demand_data.join(
    stores, on="store_id", how="inner"
).join(
    products, on="sku", how="inner"
)

serialize(spark, sales_data, path + "sales_data.parquet").show(5)

22/12/28 17:45:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/12/28 17:45:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/12/28 17:45:31 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


22/12/28 17:45:31 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


+---+--------+----------+-----+---------+--------+---------+--------+-----+----+----------+
|sku|store_id|      date|units|promo_cat|discount|region_id|subclass|class|dept|base_price|
+---+--------+----------+-----+---------+--------+---------+--------+-----+----+----------+
|102|      11|2018-01-01|    0|     null|    null|        1|       0|    0|   9|     164.0|
|102|       1|2018-01-22|    0|     null|    null|        2|       0|    0|   9|     164.0|
|102|      11|2018-01-15|    0|     null|    null|        1|       0|    0|   9|     164.0|
|102|       1|2018-02-26|    0|     null|    null|        2|       0|    0|   9|     164.0|
|102|      11|2018-03-12|    0|     null|    null|        1|       0|    0|   9|     164.0|
+---+--------+----------+-----+---------+--------+---------+--------+-----+----+----------+
only showing top 5 rows



## Feature Engineering

In [12]:
sales_data = spark.read.parquet(path + "sales_data.parquet", header="true", inferSchema="true")

engineered_data = engineerFeatures(
    data=sales_data,
    config=features_config
)

serialize(spark, engineered_data, path + "engineered_data.parquet").show(5)

+--------+---+--------+----------+-----+---------+--------+---------+-----+----+----------+--------------+----------------------+-------------+---------------+
|subclass|sku|store_id|      date|units|promo_cat|discount|region_id|class|dept|base_price|_discount_rate|_promoted_sku_subclass|_week_of_year|_promo_category|
+--------+---+--------+----------+-----+---------+--------+---------+-----+----+----------+--------------+----------------------+-------------+---------------+
|       0|102|      11|2018-01-01|    0|     null|    null|        1|    0|   9|     164.0|           1.0|                   296|            1|           null|
|       0|358|      41|2018-03-26|    0|     null|    null|        1|    0|   2|      15.0|           1.0|                   296|           13|           null|
|       0|102|       1|2018-01-22|    0|     null|    null|        2|    0|   9|     164.0|           1.0|                   296|            4|           null|
|       0|358|      37|2018-06-18|    0|

## Modelling

In [9]:
engineered_data = spark.read.parquet(path + "engineered_data.parquet", header="true", inferSchema="true")
engineered_data = engineered_data.where(F.col("sku")<400) # Just to reduce the size of the data for less memory consumption

train_data, test_data = splitData(
        data=engineered_data,
        model_config=model_config,
        features_config=features_config
    )

serialize(spark, train_data, path + "train_data.parquet").show(5)
serialize(spark, test_data, path + "test_data.parquet").show(5)

+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
|sku|subclass|store_id|region_id|_week_of_year|_promo_category|_discount_rate|_promoted_sku_subclass|units|
+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
|118|       1|      18|        1|            8|           null|           1.0|                   290|    0|
|377|       1|      11|        1|           22|            nan|           1.0|                   290|    1|
|118|       1|      24|        2|            9|           null|           1.0|                   290|    0|
|377|       1|      24|        2|            6|           null|           1.0|                   290|    0|
|118|       1|      18|        1|           12|           null|           1.0|                   290|    0|
+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
only showing top 5 rows



+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
|sku|subclass|store_id|region_id|_week_of_year|_promo_category|_discount_rate|_promoted_sku_subclass|units|
+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
|377|       1|      11|        1|           48|           null|           1.0|                   290|    0|
|377|       1|      11|        1|            5|            nan|           1.0|                   290|    1|
|377|       1|      11|        1|            6|            nan|           1.0|                   290|    1|
|377|       1|      11|        1|            7|           null|           1.0|                   290|    0|
|377|       1|      11|        1|           12|           null|           1.0|                   290|    0|
+---+--------+--------+---------+-------------+---------------+--------------+----------------------+-----+
only showing top 5 rows



In [5]:
train_data = pd.read_parquet(f"data/train_data.parquet", "pyarrow")
test_data = pd.read_parquet(f"data/test_data.parquet", "pyarrow")

# preparing the training data
X_train_ohe_sparse, y_train = prepare_data(
    train_data,
    model_config,
    features_config,
    prefix="train",
)

# preparing the inferencing data
X_test_ohe_sparse, y_test = prepare_data(
    test_data,
    model_config,
    features_config,
    prefix="test",
)

train_model(X_train_ohe_sparse, X_test_ohe_sparse, y_train, y_test, model_config)

[2022-12-30 15:26:31.081387] One-hot encoding the train dataframe
[2022-12-30 15:26:46.990414] Transforming the train one-hot encoded data into a CSR matrix
[2022-12-30 15:27:01.721415] One-hot encoding the test dataframe
[2022-12-30 15:27:10.983409] Transforming the test one-hot encoded data into a CSR matrix
[2022-12-30 15:27:18.988850] Loading the xgboost model
[2022-12-30 15:27:18.989033] Fitting the xgboost model
[2022-12-30 15:28:03.564492] Generating predictions
[2022-12-30 15:28:06.180636] mean_squared_error_ =0.11511720835514745


array([-0.00140893, -0.00140893, -0.00054343, ...,  0.00044674,
        0.00056613,  0.0083237 ], dtype=float32)

In [6]:
max_depth_list = [3,4]
learning_rate_list = [0.1, 0.001]
n_estimators_list = [20, 25]

for max_depth, learning_rate, n_estimators in zip(max_depth_list, learning_rate_list, n_estimators_list):
    model_params = {"max_depth":max_depth, "learning_rate":learning_rate, "n_estimators":n_estimators}
    MLFlow_train_model(X_train_ohe_sparse, X_test_ohe_sparse, y_train, y_test, model_config, model_params)

2022/12/30 15:28:06 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



[2022-12-30 15:28:07.130725] Logged data and model in run 554c32dc985049d9a0c20f4589d5956e
[2022-12-30 15:28:07.130880] [Run 554c32dc985049d9a0c20f4589d5956e]: Loading the xgboost model
[2022-12-30 15:28:07.130942] [Run 554c32dc985049d9a0c20f4589d5956e]: Fitting the xgboost model


2022/12/30 15:28:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


[2022-12-30 15:28:17.279357] [Run 554c32dc985049d9a0c20f4589d5956e]: Generating predictions
[2022-12-30 15:28:17.641315] [Run 554c32dc985049d9a0c20f4589d5956e]: mean_squared_error_=0.13993235249196243
[2022-12-30 15:28:20.348015] Logged data and model in run c78ca2c373904ba89a7eb3d7ab9d4da0
[2022-12-30 15:28:20.348141] [Run c78ca2c373904ba89a7eb3d7ab9d4da0]: Loading the xgboost model
[2022-12-30 15:28:20.348186] [Run c78ca2c373904ba89a7eb3d7ab9d4da0]: Fitting the xgboost model
[2022-12-30 15:28:32.317822] [Run c78ca2c373904ba89a7eb3d7ab9d4da0]: Generating predictions
[2022-12-30 15:28:32.855430] [Run c78ca2c373904ba89a7eb3d7ab9d4da0]: mean_squared_error_=0.47039567296478074


In [12]:
! mlflow ui

[2022-12-30 15:36:21 +0000] [8156] [INFO] Starting gunicorn 20.1.0
[2022-12-30 15:36:21 +0000] [8156] [INFO] Listening at: http://127.0.0.1:5000 (8156)
[2022-12-30 15:36:21 +0000] [8156] [INFO] Using worker: sync
[2022-12-30 15:36:21 +0000] [8157] [INFO] Booting worker with pid: 8157
[2022-12-30 15:36:21 +0000] [8159] [INFO] Booting worker with pid: 8159
[2022-12-30 15:36:21 +0000] [8162] [INFO] Booting worker with pid: 8162
[2022-12-30 15:36:21 +0000] [8163] [INFO] Booting worker with pid: 8163
^C
[2022-12-30 15:38:31 +0000] [8156] [INFO] Handling signal: int
[2022-12-30 15:38:31 +0000] [8159] [INFO] Worker exiting (pid: 8159)
[2022-12-30 15:38:31 +0000] [8163] [INFO] Worker exiting (pid: 8163)
[2022-12-30 15:38:31 +0000] [8162] [INFO] Worker exiting (pid: 8162)
[2022-12-30 15:38:31 +0000] [8157] [INFO] Worker exiting (pid: 8157)
